In [37]:
import customtkinter as ctk
import sqlite3

connection = sqlite3.connect('JIG-3000.db')
cursor = connection.cursor()
cursor = connection.execute('''CREATE TABLE IF NOT EXISTS "jig-3000"(
                            id INTEGER PRIMARY KEY,
                            problems TEXT,
                            objects TEXT)''')

root = ctk.CTk()
root.geometry('500x500')
root.title('JIG-3000')

custom_colors = {
                'gen1': '#00bd55',
                'gen2': '#9ec1a8',
                'bg': '',
                'fg': '',
                'none': 'transparent'
}

fonts = {
    'gen': ('Futura',25,'bold'),
    'btns': ('Arial',12,'bold'),
    'lbl': ('Arial',10,'bold'),
    'head': ('American Typewriter',25, 'bold'),
}


def center_window(root, width, height):
    screen_height = root.winfo_screenheight()
    screen_width = root.winfo_screenwidth()

    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    root.geometry(f'{width}x{height}+{x}+{y}')

root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)  # No expansion for columns

################################
#         Warning popup        #
################################

def open_popup(popup_type):
    if popup_type == 'problem':
        w_popup = ctk.CTkToplevel(master=background)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This problem already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nsew')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')

    elif popup_type == 'solution':
        w_popup = ctk.CTkToplevel(master=root)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This object already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nesw')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')

################################
#       Buttons function       #
################################

def on_button_click(button_id):
    if button_id == 'edit':
        popup = ctk.CTkToplevel(master=root)
        popup.title('Entry editor')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)
        
        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')

        label = ctk.CTkLabel(master=popup, wraplength=360, justify='left', text='Which entry do you want to edit?')
        label.grid(row=0, column=0, pady=20, padx=20, sticky='nesw', columnspan=2)

        p_button = ctk.CTkButton(master=popup, fg_color='green', text='Problem')
        s_button = ctk.CTkButton(master=popup, fg_color='green', text='Object')
        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)

        p_button.grid(row=1, column=0, padx=20, pady=(0,20))
        s_button.grid(row=1, column=1, padx=20, pady=(0,20))
        close_button.grid(row=2, column=0, padx=20, pady=(0,20))

    elif button_id == 'generate':
        cursor.execute('''SELECT objects FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        solution = cursor.fetchone()[0]
        cursor.execute('''SELECT problems FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        problem = cursor.fetchone()[0]

        label_combination.configure(text=f'Solve the problem: {problem}\n with a {solution}')

    elif button_id == 'problem':
        user_input = entry_problem.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE problems = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('problem')
            entry_problem.delete(0, ctk.END)

        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE problems IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET problems = ? WHERE id = ?''', (user_input, empty_row[0]))
            else:
                cursor.execute('''INSERT INTO "jig-3000" (problems) VALUES(?)''', (user_input,))
            
            connection.commit()
            entry_problem.delete(0, ctk.END)

    elif button_id == 'solution':
        user_input = entry_solution.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE objects = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('solution')
            entry_solution.delete(0, ctk.END)
        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE objects IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET objects = ? WHERE id = ?''', (user_input, empty_row[0]))

            else:
                cursor.execute('''INSERT INTO "jig-3000" (objects) VALUES(?)''', (user_input,))

            connection.commit()
            entry_solution.delete(0, ctk.END)

    elif button_id == 'how_to':
        popup = ctk.CTkToplevel(master=background)
        popup.geometry('600x500')
        popup.title('How to use the JIG-3000')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)

        label_summary = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15),
                            text = ("The JIG-3000 stimulates creativity by connecting unrelated problems "
                                    "and objects. Based on Arthur Koestler's bi-sociative thinking model, "
                                    "it encourages humorous or absurd solutions by pairing problems with "
                                    "random objects, breaking conventional thought patterns."
                                    ))

        label_summary.grid(row=0, column=0, pady=20, padx=20, sticky='nesw')

        label = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15,'bold'),
                            text='1. Enter a specific PROBLEM and a random OBJECT.*\n\n'\
                                '2. Press generate.\n\n'\
                                '3. Try solving the PROBLEM with the OBJECT.**')
        label.grid(row=1, column=0, pady=(0,20), padx=20, sticky='nesw')

        label_example = ctk.CTkLabel(master=popup,
                                    wraplength=400,
                                    justify='left',
                                    text=("EXAMPLE:\n"
                                    "When the PROBLEM is 'Global warming' and the OBJECT is 'Tablecloths,' you might suggest covering "
                                    "the world in white tablecloths to reflect sunlight and cool the planet. This could boost textile "
                                    "industries and avoid color clashes. To prevent interference with photosynthesis, use lace tablecloths "
                                    "and let tree branches poke through. Embrace the absurdity fully!"))

        label_example.grid(row=2, column=0, pady=(0,20), padx=20, sticky='nesw')

        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)
        close_button.grid(row=3, column=0, padx=20, pady=(0,20))

        label = ctk.CTkLabel(master=popup, 
                    wraplength=400, 
                    justify='left',
                    font=('Arial',10),
                    text='* Repeat as many times as you want. Try to keep the piles equal in numbers.\n'\
                        '** Your answer doesn\'t have to be logical, or even follow the laws of physics.')
        
        label.grid(row=4, column=0, pady=(0,20), padx=20, sticky='nesw')

        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')


background = ctk.CTkFrame(root, fg_color='#F45D3A')
background.grid_rowconfigure(0, weight=0)
for i in range(1,8):
    background.grid_rowconfigure(i, weight=1)

label_header = ctk.CTkLabel(master=background, text_color='#F45D3A', bg_color='#000000', fg_color=custom_colors['none'], font=fonts['head'], text='Welcome to the JIG-3000!')
label_combination = ctk.CTkLabel(master=background, fg_color=custom_colors['none'], font=fonts['gen'],text_color='red', text='Generate a random pair')

entry_problem = ctk.CTkEntry(master=background,text_color='black', fg_color='white', placeholder_text='Enter a specific problem...', placeholder_text_color='darkgrey')
entry_problem.bind('<Return>', lambda event: on_button_click('problem'))
entry_solution = ctk.CTkEntry(master=background,text_color='black', fg_color='white', placeholder_text='Enter a random object...', placeholder_text_color='darkgrey')
entry_solution.bind('<Return>', lambda event: on_button_click('solution'))

button_how_to = ctk.CTkButton(master=background, fg_color='green', text='How to?', command=lambda: on_button_click('how_to'))
button_edit = ctk.CTkButton(master=background, fg_color='green', text='Edit', command=lambda: on_button_click('edit'))
button_enter_1 = ctk.CTkButton(master=background, fg_color='green', text='Enter', command=lambda: on_button_click('problem'))
button_enter_2 = ctk.CTkButton(master=background, fg_color='green', text='Enter', command=lambda: on_button_click('solution'))
button_generate = ctk.CTkButton(master=background, text_color='#203826', font=fonts['gen'], fg_color=custom_colors['gen1'], text='Generate', hover_color=custom_colors['gen2'], command=lambda: on_button_click('generate'))

background.grid(row=0, column=0, sticky='nesw', columnspan=2)

label_header.grid(row=0, column=0, pady=0, padx=0, ipady=0, columnspan=2, sticky='ew')
label_combination.grid(row=6, column=0, pady=20, padx=20, columnspan=2)

entry_problem.grid(row=3, column=0, pady=20, padx=20, sticky='ew')
entry_solution.grid(row=4, column=0, pady=20, padx=20, sticky='ew')

button_how_to.grid(row=1,column=0, pady=20, padx=20)
button_edit.grid(row=5,column=1, pady=20, padx=20)
button_enter_1.grid(row=3, column=1, pady=20, padx=20)
button_enter_2.grid(row=4, column=1, pady=20, padx=20)
button_generate.grid(row=5, column=0, ipady=20, columnspan=2)

root.mainloop()

connection.close()

In [21]:
import sqlite3

database = sqlite3.connect('JIG-3000.db')
cursor = database.cursor()

for row in cursor.execute('''SELECT * FROM "jig-3000"'''):
    print(row)

database.close()

(1, 'HOW', 'Daddy')
(2, 'ABOUT', 'Buddy')
(3, 'NOW', 'OOOPS')
(4, 'YEAH', 'GEHT')
(5, 'HOLLA', 'HELLO')
(6, 'BUBBA', 'MY')
(7, 'WORLD HUNGER', 'DEAR')
(8, 'DERSAN', '')
(9, 'WERSAN', 'HERZL')
(10, 'TEST', 'HALLO')
(11, '', None)
(12, 'HALLO', None)


# Next steps

[X] Finalize the how to text/ steps

[X] Store inputs in sqlite data base

[ ] Open popup to edit entries based on index number

[ ] Design...

[X] Change Example with three steps in how to and create tags to change the fonts

[X] Connect database with the generate button 

[X] How to make sure the entries start where the first row is none and fill it up

[X] Get rid of the '' before and after the row entries

[X] When entering the problem the entrybox does not reset

[ ] Show check mark after entering new value

[ ] Enter should not work, if the entry widget is empty

In [ ]:
import customtkinter as ctk
import sqlite3

# Database setup
connection = sqlite3.connect('JIG-3000.db')
cursor = connection.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS "jig-3000"(
                            id INTEGER PRIMARY KEY,
                            problems TEXT,
                            objects TEXT)''')

# Root window setup
root = ctk.CTk()
root.geometry('500x600')
root.title('JIG-3000')

def center_window(root, width, height):
    screen_height = root.winfo_screenheight()
    screen_width = root.winfo_screenwidth()

    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    root.geometry(f'{width}x{height}+{x}+{y}')

center_window(root, 500, 600)

# Configure root grid
for i in range(10):
    root.grid_columnconfigure(i, weight=1)
    root.grid_rowconfigure(i, weight=1)

# Popup functions
def open_popup(popup_type):
    w_popup = ctk.CTkToplevel(master=root)
    w_popup.title('WARNING!')
    
    w_popup.grid_columnconfigure(0, weight=1)
    w_popup.grid_rowconfigure(0, weight=1)
    w_popup.grid_rowconfigure(1, weight=1)

    message = "This problem already exists!" if popup_type == 'problem' else "This object already exists!"
    label = ctk.CTkLabel(master=w_popup, justify='center', text=message)
    label.grid(row=0, column=0, pady=20, padx=20, sticky='nsew')

    close_button = ctk.CTkButton(master=w_popup, text='Close', command=w_popup.destroy)
    close_button.grid(row=1, column=0, padx=20, pady=(0, 20), sticky='nsew')

    w_popup.update_idletasks()
    width = w_popup.winfo_reqwidth()
    height = w_popup.winfo_reqheight()
    w_popup.geometry(f'{width}x{height}')

# Custom colors and font schemes
custom_colors = {
    "bg": "#2C2C2C",        # Dark background
    "fg": "#E0E0E0",        # Light foreground text
    "accent": "#007ACC",    # Blue accent
    "button": "#505050",    # Darker buttons
    "warning": "#FF6347"    # Warning color
}

fonts = {
    "title": ('Arial', 20, 'bold'),
    "label": ('Arial', 15),
    "small": ('Arial', 10)
}

root.configure(bg=custom_colors["bg"])

# Event handler
def on_button_click(button_id):
    if button_id == 'edit':
        popup = ctk.CTkToplevel(master=root)
        popup.title('Entry Editor')
        popup.geometry('400x300')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)
        
        label = ctk.CTkLabel(master=popup, text='Which entry do you want to edit?', wraplength=360, justify='left', font=fonts["label"])
        label.grid(row=0, column=0, pady=20, padx=20, sticky='nsew', columnspan=2)

        p_button = ctk.CTkButton(master=popup, text='Problem', command=lambda: print('Edit problem clicked'))
        s_button = ctk.CTkButton(master=popup, text='Object', command=lambda: print('Edit object clicked'))
        close_button = ctk.CTkButton(master=popup, text='Close', command=popup.destroy)

        p_button.grid(row=1, column=0, padx=20, pady=(0, 20))
        s_button.grid(row=1, column=1, padx=20, pady=(0, 20))
        close_button.grid(row=2, column=0, padx=20, pady=(0, 20), columnspan=2)

    elif button_id == 'generate':
        cursor.execute('''SELECT objects FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        solution = cursor.fetchone()[0]
        cursor.execute('''SELECT problems FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        problem = cursor.fetchone()[0]

        label_combination.configure(text=f'Solve the problem: {problem}\n with a {solution}')

    elif button_id == 'problem':
        user_input = entry_problem.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE problems = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('problem')
            entry_problem.delete(0, ctk.END)
        else:
            cursor.execute('''INSERT INTO "jig-3000" (problems) VALUES(?)''', (user_input,))
            connection.commit()
            entry_problem.delete(0, ctk.END)

    elif button_id == 'solution':
        user_input = entry_solution.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE objects = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('solution')
            entry_solution.delete(0, ctk.END)
        else:
            cursor.execute('''INSERT INTO "jig-3000" (objects) VALUES(?)''', (user_input,))
            connection.commit()
            entry_solution.delete(0, ctk.END)

    elif button_id == 'how_to':
        popup = ctk.CTkToplevel(master=root)
        popup.geometry('500x400')
        popup.title('How to use the JIG-3000')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)
        popup.grid_rowconfigure(2, weight=1)

        label_summary = ctk.CTkLabel(master=popup, wraplength=400, justify='left', font=fonts["label"],
                                     text=("The JIG-3000 stimulates creativity by connecting unrelated problems "
                                           "and objects. Based on Arthur Koestler's bi-sociative thinking model, "
                                           "it encourages humorous or absurd solutions by pairing problems with "
                                           "random objects, breaking conventional thought patterns."))
        label_summary.grid(row=0, column=0, pady=20, padx=20, sticky='nsew')

        label = ctk.CTkLabel(master=popup, wraplength=400, justify='left', font=fonts["label"],
                             text='1. Enter a specific PROBLEM and a random OBJECT.*\n\n'
                                  '2. Press generate.\n\n'
                                  '3. Try solving the PROBLEM with the OBJECT.**')
        label.grid(row=1, column=0, pady=(0, 20), padx=20, sticky='nsew')

        label_example = ctk.CTkLabel(master=popup, wraplength=400, justify='left', text=("EXAMPLE:\n"
                                                                                         "When the PROBLEM is 'Global warming' and the OBJECT is 'Tablecloths,' you might suggest covering "
                                                                                         "the world in white tablecloths to reflect sunlight and cool the planet. This could boost textile "
                                                                                         "industries and avoid color clashes. To prevent interference with photosynthesis, use lace tablecloths "
                                                                                         "and let tree branches poke through. Embrace the absurdity fully!"))

        label_example.grid(row=2, column=0, pady=(0, 20), padx=20, sticky='nsew')

        close_button = ctk.CTkButton(master=popup, text='Close', command=popup.destroy)
        close_button.grid(row=3, column=0, padx=20, pady=(0, 20))

        label_disclaimer = ctk.CTkLabel(master=popup, wraplength=400, justify='left', font=fonts["small"],
                                        text='* Repeat as many times as you want. Try to keep the piles equal in numbers.\n'
                                             '** Your answer doesn\'t have to be logical, or even follow the laws of physics.')
        label_disclaimer.grid(row=4, column=0, pady=(0, 20), padx=20, sticky='nsew')

        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')

# Widgets
label_welcome = ctk.CTkLabel(master=root, text='Welcome to the JIG-3000!', font=fonts["title"], fg_color=custom_colors["bg"], text_color=custom_colors["fg"])
label_combination = ctk.CTkLabel(master=root, text='Du hast folgendes gewählt', font=fonts["title"], fg_color=custom_colors["bg"], text_color='red')

entry_problem = ctk.CTkEntry(master=root, fg_color=custom_colors["accent"], placeholder_text='Enter a specific problem...', placeholder_text_color=custom_colors["bg"])
entry_problem.bind('<Return>', lambda event: on_button_click('problem'))
entry_solution = ctk.CTkEntry(master=root, fg_color=custom_colors["accent"], placeholder_text='Enter a random object...', placeholder_text_color=custom_colors["bg"])
entry_solution.bind('<Return>', lambda event: on_button_click('solution'))

button_how_to = ctk.CTkButton(master=root, fg_color=custom_colors["button"], text='How to?', command=lambda: on_button_click('how_to'))
button_edit = ctk.CTkButton(master=root, fg_color=custom_colors["button"], text='Edit', command=lambda: on_button_click('edit'))
button_enter_1 = ctk.CTkButton(master=root, fg_color=custom_colors["accent"], text='Enter', command=lambda: on_button_click('problem'))
button_enter_2 = ctk.CTkButton(master=root, fg_color=custom_colors["accent"], text='Enter', command=lambda: on_button_click('solution'))
button_generate = ctk.CTkButton(master=root, fg_color=custom_colors["warning"], text='Generate', command=lambda: on_button_click('generate'))

# Layout
label_welcome.grid(row=0, column=0, pady=20, padx=20, columnspan=2)
label_combination.grid(row=1, column=0, pady=20, padx=20, columnspan=2)

entry_problem.grid(row=2, column=0, pady=20, padx=20, sticky='ew')
entry_solution.grid(row=3, column=0, pady=20, padx=20, sticky='ew')

button_enter_1.grid(row=2, column=1, pady=20, padx=20)
button_enter_2.grid(row=3, column=1, pady=20, padx=20)

button_generate.grid(row=4, column=0, pady=20, padx=20, columnspan=2)

button_how_to.grid(row=5, column=0, pady=20, padx=20)
button_edit.grid(row=5, column=1, pady=20, padx=20)

root.mainloop()

connection.close()


In [38]:
from tkinter import *
from tkinter import font

root = Tk()
root.title('Font Families')
fonts=list(font.families())
fonts.sort()

def populate(frame):
    '''Put in the fonts'''
    listnumber = 1
    for i, item in enumerate(fonts):
        label = "listlabel" + str(listnumber)
        label = Label(frame,text=item,font=(item, 16))
        label.grid(row=i)
        label.bind("<Button-1>",lambda e,item=item:copy_to_clipboard(item))
        listnumber += 1

def copy_to_clipboard(item):
    root.clipboard_clear()
    root.clipboard_append("font=('" + item.lstrip('@') + "', 12)")

def onFrameConfigure(canvas):
    '''Reset the scroll region to encompass the inner frame'''
    canvas.configure(scrollregion=canvas.bbox("all"))

canvas = Canvas(root, borderwidth=0, background="#ffffff")
frame = Frame(canvas, background="#ffffff")
vsb = Scrollbar(root, orient="vertical", command=canvas.yview)
canvas.configure(yscrollcommand=vsb.set)

vsb.pack(side="right", fill="y")
canvas.pack(side="left", fill="both", expand=True)
canvas.create_window((4,4), window=frame, anchor="nw")

frame.bind("<Configure>", lambda event, canvas=canvas: onFrameConfigure(canvas))

populate(frame)

root.mainloop()

# THIS IS THE CODE

In [1]:
import customtkinter as ctk
import sqlite3
from tkinter import messagebox

connection = sqlite3.connect('JIG-3000.db')
cursor = connection.cursor()
cursor = connection.execute('''CREATE TABLE IF NOT EXISTS "jig-3000"(
                            id INTEGER PRIMARY KEY,
                            problems TEXT,
                            objects TEXT)''')

root = ctk.CTk()
root.geometry('500x500')
root.title('JIG-3000')
root.resizable(False, False)

custom_colors = {
                'gen1': '#05AFD3',
                'gen2': '#2FCCEB',
                'bg': '',
                'fg': '#44109F',
                'btns': '#05AFD3',
                'none': 'transparent'
}

fonts = {
    'gen': ('Rockwell',17,'bold'),
    'btns': ('Krungthep',15),
    'lbl': ('Arial',10,'bold'),
    'head': ('Rockwell',35, 'bold'),
}

def center_window(root, width, height):
    screen_height = root.winfo_screenheight()
    screen_width = root.winfo_screenwidth()

    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    root.geometry(f'{width}x{height}+{x}+{y}')

root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)  # No expansion for columns

################################
#         Warning popup        #
################################

def open_messeagebox(warning_type):
    if warning_type == 'problem':
        messagebox.showwarning(message='This problem already exists!')
    elif warning_type == 'object':
        messagebox.showwarning(message='This object already exists!')

################################
#       Buttons function       #
################################

def on_button_click(button_id):
    if button_id == 'edit':
        popup = ctk.CTkToplevel(master=root)
        popup.title('Editor')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)
        
        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')

        lbox = ctk.CTkScrollableFrame(popup)
        lbox.grid(row=3, column=0, pady=5, padx=5)
        
        cursor.execute('''SELECT * FROM "jig-3000"''')
        rows = cursor.fetchall()
        for row in rows:
            entry_label = ctk.CTkLabel(lbox, text=f'{row[0]}. {row[1]} {row[2]}')
            entry_label.pack(pady=5, padx=5, anchor='w')  # Pack each entry into the frame
        

        label = ctk.CTkLabel(master=popup, wraplength=360, justify='left', text='Which entry do you want to edit?')
        label.grid(row=0, column=0, pady=20, padx=20, sticky='nesw', columnspan=2)

        button_save = ctk.CTkButton(master=popup, fg_color=custom_colors['btns'], text='SAVE')
        s_button = ctk.CTkButton(master=popup, fg_color='green', text='Object')
        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)

        button_save.grid(row=1, column=0, padx=20, pady=(0,20))
        s_button.grid(row=1, column=1, padx=20, pady=(0,20))
        close_button.grid(row=2, column=0, padx=20, pady=(0,20))

        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')

    elif button_id == 'generate':
        cursor.execute('''SELECT objects FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        object = cursor.fetchone()[0]
        cursor.execute('''SELECT problems FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        problem = cursor.fetchone()[0]
        label_output.configure(text=f'PROBLEM:{problem} OBJECT: {object}')

    elif button_id == 'problem':
        user_input = entry_problem.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE problems = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_messeagebox('problem')
            entry_problem.delete(0, ctk.END)

        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE problems IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET problems = ? WHERE id = ?''', (user_input, empty_row[0]))
            else:
                cursor.execute('''INSERT INTO "jig-3000" (problems) VALUES(?)''', (user_input,))
            
            connection.commit()
            entry_problem.delete(0, ctk.END)

    elif button_id == 'object':
        user_input = entry_object.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE objects = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_messeagebox('object')
            entry_object.delete(0, ctk.END)
        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE objects IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET objects = ? WHERE id = ?''', (user_input, empty_row[0]))

            else:
                cursor.execute('''INSERT INTO "jig-3000" (objects) VALUES(?)''', (user_input,))

            connection.commit()
            entry_object.delete(0, ctk.END)

    elif button_id == 'faq':
        popup = ctk.CTkToplevel(master=background)
        popup.geometry('600x500')
        popup.title('How to use the JIG-3000')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)

        label_summary = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15),
                            text = ("The JIG-3000 stimulates creativity by connecting unrelated problems "
                                    "and objects. Based on Arthur Koestler's bi-sociative thinking model, "
                                    "it encourages humorous or absurd objects by pairing problems with "
                                    "random objects, breaking conventional thought patterns."
                                    ))
        label_summary.grid(row=0, column=0, pady=20, padx=20, sticky='nesw')

        label = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15,'bold'),
                            text='1. Enter a specific PROBLEM and a random OBJECT.*\n\n'\
                                '2. Press generate.\n\n'\
                                '3. Try solving the PROBLEM with the OBJECT.**')
        label.grid(row=1, column=0, pady=(0,20), padx=20, sticky='nesw')

        label_example = ctk.CTkLabel(master=popup,
                                    wraplength=400,
                                    justify='left',
                                    text=("EXAMPLE:\n"
                                    "When the PROBLEM is 'Global warming' and the OBJECT is 'Tablecloths,' you might suggest covering "
                                    "the world in white tablecloths to reflect sunlight and cool the planet. This could boost textile "
                                    "industries and avoid color clashes. To prevent interference with photosynthesis, use lace tablecloths "
                                    "and let tree branches poke through. Embrace the absurdity fully!"))

        label_example.grid(row=2, column=0, pady=(0,20), padx=20, sticky='nesw')

        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)
        close_button.grid(row=3, column=0, padx=20, pady=(0,20))

        label = ctk.CTkLabel(master=popup, 
                    wraplength=400, 
                    justify='left',
                    font=('Arial',10),
                    text='* Repeat as many times as you want. Try to keep the piles equal in numbers.\n'\
                        '** Your answer doesn\'t have to be logical, or even follow the laws of physics.')
        
        label.grid(row=4, column=0, pady=(0,20), padx=20, sticky='nesw')

        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')
    
    elif button_id == 'quit':
        connection.close()  # Close the SQLite connection
        root.destroy() 

background = ctk.CTkFrame(root, fg_color='#7E30FA')
for i in range(7):
    background.grid_rowconfigure(i, weight=1)
for j in range(2):
    background.grid_columnconfigure(j, weight=1)

label_header = ctk.CTkLabel(master=background, text_color='white', fg_color=custom_colors['none'], font=fonts['head'], text='JIG-3000')

label_frame = ctk.CTkFrame(master=background,fg_color=custom_colors['fg'])
label_frame.grid_columnconfigure(0,weight=1)
label_frame.grid_rowconfigure(0,weight=1)

label_output = ctk.CTkLabel(master=label_frame, fg_color=custom_colors['none'], font=fonts['lbl'],text_color='white', text='Generate a random pair')

entry_frame = ctk.CTkFrame(background, fg_color=custom_colors['fg'])
for i in range(3):
    entry_frame.grid_columnconfigure(i,weight=1)
for j in range(2):
    entry_frame.grid_rowconfigure(j,weight=1)

entry_problem = ctk.CTkEntry(master=entry_frame, text_color='black', height=35,width=230, fg_color='white', placeholder_text='Enter a problem', placeholder_text_color='darkgrey')
entry_problem.bind('<Return>', lambda event: on_button_click('problem'))
entry_object = ctk.CTkEntry(master=entry_frame, text_color='black', height=35, width=230, fg_color='white', placeholder_text='Enter a object', placeholder_text_color='darkgrey')
entry_object.bind('<Return>', lambda event: on_button_click('object'))

button_faq = ctk.CTkButton(master=background, font=fonts['btns'], text='FAQ', text_color='white', fg_color=custom_colors['gen1'], hover_color=custom_colors['gen2'], command=lambda: on_button_click('faq'))
button_edit = ctk.CTkButton(master=background,font=fonts['btns'], text='EDIT', text_color='white', fg_color=custom_colors['gen1'], hover_color=custom_colors['gen2'], command=lambda: on_button_click('edit'))
button_enter_1 = ctk.CTkButton(master=entry_frame, font=fonts['btns'], text='ADD PROBLEM', text_color='white', fg_color=custom_colors['gen1'], hover_color=custom_colors['gen2'], command=lambda: on_button_click('problem'))
button_enter_2 = ctk.CTkButton(master=entry_frame, font=fonts['btns'], text='ADD OBJECT', text_color='white', fg_color=custom_colors['gen1'], hover_color=custom_colors['gen2'],command=lambda: on_button_click('object'))
button_generate = ctk.CTkButton(master=background, border_spacing=15, corner_radius=250, border_width=5, border_color=custom_colors['fg'],text_color=custom_colors['fg'], font=fonts['gen'], fg_color='white', text='GENERATE NEW PAIR', hover_color='#E2CDB4', command=lambda: on_button_click('generate'))
button_quit = ctk.CTkButton(master=background, font=fonts['btns'], text='QUIT', text_color='white', fg_color=custom_colors['gen1'], hover_color=custom_colors['gen2'], command=lambda: on_button_click('quit'))

background.grid(row=0, column=0, pady=15, padx=15, sticky='nesw')

background.grid_rowconfigure(0, weight=0)
for i in range(1,4):
    background.grid_rowconfigure(i, weight=1)
for j in range(4):
    background.grid_columnconfigure(j,weight=1)

label_header.grid(row=0, column=0, pady=(15,0), padx=10, ipady=10, columnspan=4, sticky='ew')
label_frame.grid(row=3,column=0, columnspan=4, sticky='ew', padx=15)
label_output.grid(row=0, column=0, pady=10, padx=20, columnspan=4, sticky='ew')

entry_frame.grid(row=1,column=0,columnspan=4, padx=15, pady=(20,0), sticky='nesw')
entry_problem.grid(row=0, column=0, pady=(20,5), padx=(20,0), ipady=5, sticky='nesw', columnspan=2)
entry_object.grid(row=1, column=0, pady=(5,20), padx=(20,0), ipady=5, sticky='nesw', columnspan=2)

button_faq.grid(row=4, column=0, pady=20, padx=20, ipady=5, sticky='nesw')
button_edit.grid(row=4, column=1, pady=20, padx=20, ipady=5, sticky='nesw')
button_enter_1.grid(row=0, column=2, pady=(20,5), padx=(0,20), ipady=5, sticky='e')
button_enter_2.grid(row=1, column=2, pady=(5,20), padx=(0,20), ipady=5, sticky='e')
button_generate.grid(row=2, column=0, pady=(20), padx=100, columnspan=4, sticky='ew')
button_quit.grid(row=4,column=2, pady=20, padx=20, ipady=5, sticky='nesw')

root.mainloop()


In [55]:
import customtkinter as ctk
import sqlite3

# Database connection and table creation
connection = sqlite3.connect('JIG-3000.db')
cursor = connection.cursor()
cursor = connection.execute('''CREATE TABLE IF NOT EXISTS "jig-3000"(
                            id INTEGER PRIMARY KEY,
                            problems TEXT,
                            objects TEXT)''')

# Main window configuration
root = ctk.CTk()
root.geometry('500x500')
root.title('JIG-3000')

# Custom colors and fonts
custom_colors = {
    'gen1': '#E3B747',
    'gen2': '#E1C479',
    'bg': '',
    'fg': '',
    'btns': '#460F9E',
    'none': 'transparent'
}

fonts = {
    'gen': ('Futura', 25, 'bold'),
    'btns': ('Krungthep', 18),
    'lbl': ('Arial', 10, 'bold'),
    'head': ('American Typewriter', 25, 'bold'),
}

# Center window function
def center_window(root, width, height):
    screen_height = root.winfo_screenheight()
    screen_width = root.winfo_screenwidth()

    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    root.geometry(f'{width}x{height}+{x}+{y}')

root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)

################################
#         Warning popup        #
################################

def open_popup(popup_type):
    if popup_type == 'problem':
        w_popup = ctk.CTkToplevel(master=background)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This problem already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nsew')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')

    elif popup_type == 'solution':
        w_popup = ctk.CTkToplevel(master=root)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This object already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nesw')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')

################################
#       Buttons function       #
################################

def on_button_click(button_id):
    if button_id == 'edit':
        popup = ctk.CTkToplevel(master=root)
        popup.title('Entry editor')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)
        
        popup.update_idletasks()
        width = popup.winfo_reqwidth()
        height = popup.winfo_reqheight()
        popup.geometry(f'{width}x{height}')

        label = ctk.CTkLabel(master=popup, wraplength=360, justify='left', text='Which entry do you want to edit?')
        label.grid(row=0, column=0, pady=20, padx=20, sticky='nesw', columnspan=2)

        p_button = ctk.CTkButton(master=popup, fg_color='green', text='Problem')
        s_button = ctk.CTkButton(master=popup, fg_color='green', text='Object')
        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)

        p_button.grid(row=1, column=0, padx=20, pady=(0,20))
        s_button.grid(row=1, column=1, padx=20, pady=(0,20))
        close_button.grid(row=2, column=0, padx=20, pady=(0,20))

    elif button_id == 'generate':
        cursor.execute('''SELECT objects FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        solution = cursor.fetchone()[0]
        cursor.execute('''SELECT problems FROM "jig-3000" ORDER BY RANDOM() LIMIT 1''')
        problem = cursor.fetchone()[0]

        label_combination.configure(text=f'Solve the problem: {problem}\n with a {solution}')

    elif button_id == 'problem':
        user_input = entry_problem.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE problems = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('problem')
            entry_problem.delete(0, ctk.END)

        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE problems IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET problems = ? WHERE id = ?''', (user_input, empty_row[0]))
            else:
                cursor.execute('''INSERT INTO "jig-3000" (problems) VALUES(?)''', (user_input,))
            
            connection.commit()
            entry_problem.delete(0, ctk.END)

    elif button_id == 'solution':
        user_input = entry_solution.get().strip().upper()
        cursor.execute('''SELECT COUNT(*) FROM "jig-3000" WHERE objects = ?''', (user_input,))
        entry_check = cursor.fetchone()[0]

        if entry_check > 0:
            open_popup('solution')
            entry_solution.delete(0, ctk.END)
        else:
            cursor.execute('''SELECT id FROM "jig-3000" WHERE objects IS NULL LIMIT 1''')
            empty_row = cursor.fetchone()

            if empty_row:
                cursor.execute('''UPDATE "jig-3000" SET objects = ? WHERE id = ?''', (user_input, empty_row[0]))

            else:
                cursor.execute('''INSERT INTO "jig-3000" (objects) VALUES(?)''', (user_input,))

            connection.commit()
            entry_solution.delete(0, ctk.END)

    elif button_id == 'how_to':
        popup = ctk.CTkToplevel(master=background)
        popup.geometry('600x500')
        popup.title('How to use the JIG-3000')

        popup.grid_columnconfigure(0, weight=1)
        popup.grid_rowconfigure(0, weight=1)
        popup.grid_rowconfigure(1, weight=1)

        label_summary = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15),
                            text = ("The JIG-3000 stimulates creativity by connecting unrelated problems "
                                    "and objects. Based on Arthur Koestler's bi-sociative thinking model, "
                                    "it encourages humorous or absurd solutions by pairing problems with "
                                    "random objects, breaking conventional thought patterns."
                                    ))

        label_summary.grid(row=0, column=0, pady=20, padx=20, sticky='nesw')

        label = ctk.CTkLabel(master=popup, 
                            wraplength=400, 
                            justify='left',
                            font=('Arial',15,'bold'),
                            text='1. Enter a specific PROBLEM and a random OBJECT.*\n\n'\
                                '2. Press generate.\n\n'\
                                '3. Try solving the PROBLEM with the OBJECT.**')
        label.grid(row=1, column=0, pady=(0,20), padx=20, sticky='nesw')

        label_example = ctk.CTkLabel(master=popup,
                                    wraplength=400,
                                    justify='left',
                                    text=("EXAMPLE:\n"
                                    "When the PROBLEM is 'Global warming' and the OBJECT is 'Tablecloths,' you might suggest covering "
                                    "the world in white tablecloths to reflect sunlight and cool the planet. This could boost textile "
                                    "industries and avoid color clashes. To prevent interference with photosynthesis, use lace tablecloths "
                                    "and let tree branches poke through. Embrace the absurdity fully!"))

        label_example.grid(row=2, column=0, pady=(0,10), padx=20, sticky='nsew')

        close_button = ctk.CTkButton(master=popup, fg_color='blue', text='Quit', command=popup.destroy)
        close_button.grid(row=3, column=0, padx=20, pady=(0,20), sticky='nsew')

    elif button_id == 'quit':
        root.destroy()

################################
#       Interface setup        #
################################

background = ctk.CTkFrame(master=root)
background.grid(row=0, column=0, sticky='nesw', padx=20, pady=20)

title = ctk.CTkLabel(master=background, 
                    font=('American Typewriter', 50, 'bold'),
                    justify='center',
                    text="JIG-3000")

title.grid(row=0, column=0, padx=20, pady=20, sticky='n')

# Frame for entry fields
entry_frame = ctk.CTkFrame(master=background)
entry_frame.grid(row=1, column=0, padx=20, pady=20, sticky='nsew')

entry_problem = ctk.CTkEntry(master=entry_frame,
                             width=300,
                             height=30,
                             placeholder_text='Enter the problem')

entry_problem.grid(row=0, column=0, padx=10, pady=10, sticky='ew')

btn_problem = ctk.CTkButton(master=entry_frame,
                            text='Add problem',
                            command=lambda: on_button_click('problem'))

btn_problem.grid(row=0, column=1, padx=10, pady=10, sticky='ew')

entry_solution = ctk.CTkEntry(master=entry_frame,
                              width=300,
                              height=30,
                              placeholder_text='Enter the object')

entry_solution.grid(row=1, column=0, padx=10, pady=10, sticky='ew')

btn_solution = ctk.CTkButton(master=entry_frame,
                             text='Add object',
                             command=lambda: on_button_click('solution'))

btn_solution.grid(row=1, column=1, padx=10, pady=10, sticky='ew')

btn_generate = ctk.CTkButton(master=background,
                             text='Generate new JIG',
                             command=lambda: on_button_click('generate'))

btn_generate.grid(row=2, column=0, padx=20, pady=20, sticky='n')

label_combination = ctk.CTkLabel(master=background,
                                 font=('Arial', 15),
                                 text='Solve the problem:')
label_combination.grid(row=3, column=0, padx=20, pady=20, sticky='n')

btn_how_to = ctk.CTkButton(master=background,
                           fg_color='blue',
                           text='How to use the JIG-3000',
                           command=lambda: on_button_click('how_to'))

btn_how_to.grid(row=4, column=0, padx=20, pady=(10, 10), sticky='ew')

btn_quit = ctk.CTkButton(master=background,
                         fg_color='blue',
                         text='Quit',
                         command=lambda: on_button_click('quit'))

btn_quit.grid(row=5, column=0, padx=20, pady=20, sticky='ew')

root.mainloop()


# FAQ

In [1]:
import customtkinter as ctk
import tkinter as tk

root = ctk.CTk()
root.geometry('400x400')

def onClick(button_id):
    if button_id == 'faq':
        faq = ctk.CTkToplevel(root,fg_color='blue')
        faq.title('FAQ')
        faq.geometry('400x400')

        for i in range(1):
            faq.grid_columnconfigure(i,weight=1)
        for j in range(4):
            faq.grid_rowconfigure(j,weight=1)

        btn1 = ctk.CTkButton(faq, text='1. What is the JIG-3000?')
        btn1.grid(row=0,column=0,sticky='ew', padx=25, pady=(20,10))

        btn2 = ctk.CTkButton(faq, text='2. How do I get started?')
        btn2.grid(row=1,column=0,sticky='ew', padx=25, pady=(10))

        btn3 = ctk.CTkButton(faq, text='3. Do you have an example?')
        btn3.grid(row=2,column=0,sticky='ew', padx=25, pady=(10))

        btn4 = ctk.CTkButton(faq, text='4. What is the JIG-3000?')
        btn4.grid(row=3,column=0,sticky='ew', padx=25, pady=(10,20))


root.grid_columnconfigure(0,weight=1)
root.grid_rowconfigure(0,weight=1)

btn1 = ctk.CTkButton(root, fg_color='yellow', text='EDIT', command=lambda: onClick('faq'))
btn1.grid(row=0,column=0)

root.mainloop()

In [ ]:
def open_popup(popup_type):
    if popup_type == 'problem':
        w_popup = ctk.CTkToplevel(master=background)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This problem already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nsew')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')

    elif popup_type == 'object':
        w_popup = ctk.CTkToplevel(master=root)
        w_popup.title('WARNING!')

        w_popup.grid_columnconfigure(0, weight=1)
        w_popup.grid_rowconfigure(0, weight=1)
        w_popup.grid_rowconfigure(1, weight=1)

        label = ctk.CTkLabel(master=w_popup, justify='center', text='This object already exists!')
        label.grid(row=0, column=0, pady=10, padx=20, sticky='nesw')

        close_button = ctk.CTkButton(master=w_popup, fg_color='blue', text='Quit', command=w_popup.destroy)
        close_button.grid(row=1, column=0, padx=20, pady=(0,10), sticky='nsew')

        w_popup.update_idletasks()
        width = w_popup.winfo_reqwidth()
        height = w_popup.winfo_reqheight()
        w_popup.geometry(f'{width}x{height}')


invalid command name "4378160320update"
    while executing
"4378160320update"
    ("after" script)
invalid command name "4399494912check_dpi_scaling"
    while executing
"4399494912check_dpi_scaling"
    ("after" script)
invalid command name "4399494848_click_animation"
    while executing
"4399494848_click_animation"
    ("after" script)


: 